In [1]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
predictors = {
    90 : 'General Health',
    162 : 'Marital Status',
    177 : 'Employment Status',
    (183, 184, 185, 186) : 'Weight',
    (187, 188, 189, 190) : 'Height',
    191:'Pregnancy Status',
    1457:'Metropolitan Status Code',
    1947:'Computed Physical Health Status',
    2054:'Computed Smoking Status',
    2056:'Computed E-cigarette User Status',
    270:'Do Any High Risk Situations Apply(HIV/AIDS)',
    176:'Veteran',
    163:'Education level',
    198:'Smoked at least 100 cigarettes ',
    (224, 225, 226) :'Fries',
    (236, 237, 238) : 'Cardio',
    125:'Sex',
    684:'Sex ori',
    97:'Health Coverage',
    (209, 210):'Drinking',
    106:'Heart attack',
    107:'Coronary heart disease',
    108:'Stroke',
    109:'Asthma',
    111:'Skin cancer',
    112:'Other cancer',
    113:'Pulmonary',
    114:'Arthritis',
    115:'Depression',
    116:'Kidney disease',
    117:'Diabetes'
}

In [3]:
data = []
with open('BRFSS2017.txt') as f:
    for line in f:
        col = []
        for key in predictors.keys():
            if type(key)==tuple:
                key = list(key)
                val = line[key[0]-1:key[len(key)-1]] 
            else:
                val = line[key-1]
            col.append(val)
        data.append(col)                

In [4]:
df = pd.DataFrame(np.array(data),columns = predictors.values())
orig = df.copy()

In [5]:
df.Drinking = pd.to_numeric(df.Drinking, errors='coerce').apply(lambda x: np.nan if x == 77 or x == 99 else x/4.345)
df.Drinking = df.Drinking.fillna(df.Drinking.mean())

In [6]:
df = df.apply(lambda x: pd.to_numeric(x,errors= 'coerce', downcast="float"))

In [7]:
df.loc[:,'Weight'] = df['Weight'].apply(lambda x: (x-9000)*2.205 if (x >= 9000 and x<9999) else x)
mean_weight = np.mean(df['Weight'][df['Weight'] < 7000])
df.loc[:,'Weight'] = df['Weight'].apply(lambda x: mean_weight if (x == 9999 or x==7777) else x)
df.loc[:,'Weight'] =df['Weight'].fillna(mean_weight)

df.loc[:,'Height'] = df['Height'].apply(lambda x: (x//100) * 12 +x%100 if (x <1000) else x)
df.loc[:,'Height'] = df['Height'].apply(lambda x: (x-9000) * .393701 if (x < 9999 and x>9000) else x)
mean_height = np.mean(df['Height'][df['Height'] <7000])
df.loc[:,'Height'] = df['Height'].apply(lambda x: mean_height if (x == 9999 or x==7777) else x)
df.loc[:,'Height'] =df['Height'].fillna(mean_height)

In [8]:
def to_days(i):
    a=0
    if i<200:
        a = (i-100)*7
    elif i<300:
        a = (i-200)
    elif i==300 or i==555:
        a = 0
    elif i<400:
        a = (i-300)/4.345
    return a

df.loc[:,'Fries'] = df['Fries'].apply(to_days)
mean_french = np.mean(df['Fries'][df['Fries'] < 600])
df.loc[:,'Fries'] = df['Fries'].fillna(mean_french)
df.loc[:,'Fries'] = df['Fries'].apply(lambda x: mean_french if(x==777 or x==999) else x)

In [9]:
def to_weeks(i):
    a=0
    if i<200:
        a = i-100
    elif i<300:
        a = (i-200)/4.345
    return a
df.loc[:,'Cardio'] = df['Cardio'].apply(to_weeks)
mean_cardio = np.mean(df['Cardio'][df['Cardio'] < 400])
df.loc[:,'Cardio'] = df['Cardio'].fillna(mean_cardio)
df.loc[:,'Cardio'] = df['Cardio'].apply(lambda x: mean_cardio if(x==777 or x==999) else x)

In [10]:
diseases = ['Heart attack', 'Coronary heart disease', 'Stroke', 'Asthma',
       'Skin cancer', 'Other cancer', 'Pulmonary', 'Arthritis', 'Depression',
       'Kidney disease']

def get_disease_value(val):
    if val == 7.0 or val == 9.0:
        return np.nan
    return 2 - val

def diabetes_value(val):
    if val == 7.0 or val == 9.0:
        return np.nan
    if val == 4.0:
        if np.random.random() < 0.5:
            return 1
        return 0
    return 1 if val != 3 else 0

df[diseases] = df[diseases].applymap(get_disease_value)
df['Diabetes'] = df['Diabetes'].apply(diabetes_value)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [12]:
columns = df.columns
exclude = ['Weight', 'Height', 'Cardio', 'Fries', 'Drinking']
columns = columns.values.tolist()
columns = [x for x in columns if not x in exclude and x not in diseases + ['Diabetes']]
df[columns] = df[columns].applymap(lambda x: str(x) if str(x) != '9.0' else 'nan')
one_hot = pd.get_dummies(df[columns])
full_df = df[exclude].join(df[diseases]).join(df[['Diabetes']]).join(one_hot)

In [13]:
split_idx = int(0.99 * len(full_df))
scaler = StandardScaler()
#scaler.fit(full_df[])

std_df = full_df.copy()
#std_df.loc[:, :] = scaler.transform(full_df)
trunc_std_df = std_df.iloc[:split_idx]
trunc_df = full_df.iloc[:split_idx]

In [14]:
probas = []
clfs = []
relcols = [x for x in std_df.columns if x not in diseases and x != 'Diabetes']
scaler.fit(full_df[relcols])
#std_df.loc[:, :] = scaler.transform(full_df)
X = trunc_std_df.dropna().loc[: , relcols].values
X = scaler.transform(X)
test_X = std_df.iloc[split_idx:].dropna().loc[:, relcols].values
diseases.append("Diabetes")
for disease in diseases:
    print(disease)
    pred_col = disease
    y = trunc_df.dropna()[pred_col].values
    test_y = full_df.iloc[split_idx:].dropna()[pred_col].values
    clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X, y)
    clfs.append(clf)
    proba = clf.predict_proba(X)[:, 1]
    probas.append(proba)

/Users/royluo/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


Heart attack
Coronary heart disease
Stroke
Asthma
Skin cancer
Other cancer
Pulmonary
Arthritis
Depression
Kidney disease
Diabetes


In [15]:
sorted(list(zip(clf.coef_[0], [x for x in trunc_std_df.columns.values if x in relcols])))[-1:-11:-1]
#top ten risk factors

[(0.4983614091649132, 'Weight'),
 (0.28725923380597806, 'General Health_4.0'),
 (0.23633411818326272, 'General Health_5.0'),
 (0.16779921861239658, 'Employment Status_7.0'),
 (0.158551932609696, 'General Health_3.0'),
 (0.08931532757473928, 'Pregnancy Status_nan'),
 (0.0888366041180042, 'Employment Status_8.0'),
 (0.049451992288451874, 'Education level_2.0'),
 (0.04681220823915083, 'Computed E-cigarette User Status_4.0'),
 (0.04486945040762005, 'Marital Status_3.0')]

In [16]:
%%html
<img src = "background.png", width = 1200, height = 900>

In [17]:
questions=["Weight in pounds: ",
    "Height in inches: ",
    "Exercise frequency each week (i.e. 3 times a week -> 3) : ",
    "Weekly french fries consumption frequency (i.e. 2 times a week -> 2): ",
    "Weekly alcohol consumption(i.e. 4 times a week -> 4): ",
    "Self-assessment of your health on a scale of 1 to 5, 1 being excellent: ",
    "Marital status (1: Married, 2: Divorced, 3: Widowed, 4: Separated, 5: Never married, 6: Member of an unmarried couple): ",
    "Employment status (1: Employed for wages, 2: Self-employed, 3: Out of work for 1 or more years, 4: Out of work for less than 1 year, 5: A homemaker, 6: A student, 7: Retired, 8: Unable to work, 9: Retired): ",
    "Pregnancy Status (1: Yes, 2: No): ",
    "Metropolitan Status (1: In the center of city of an MSA, 2: Outside the center city of an MSA but inside the county containing the center city, 3: Inside a suburban county of the MSA, 5: Not in an MSA ***MSA is metropolitan statistical area (region with high population density)): ",
    "Physical health status for past month (1: Zero days when physical health not good, 2: 1-13 days when physical health not good, 3: 14+ days when physical health not good): ",
    "Smoking status (1: daily smoker, 2: current smoker - now smokes some days, 3: Former smoker, 4: Never smoked): ",
    "E-cigarette status (1: daily usage, 2: current user - uses some days, 3: Former E-cigarette user, 4: Never used E-cigarettes): ",
    "HIV/AIDS risk factor (1: If any of the following applies - You have injected any drug other than those prescribed for you in the past year. You have been treated for a sexually transmitted disease or STD in the past year. You have given or received money or drugs in exchange for sex in the past year., 2: otherwise): ",
    "Health care coverage (1: Yes (insurance, prepaid plans, or health service), 2: No): ",
    "Enter your sex (1: Male, 2: Female): ",
    "Veteran status (1: Yes, 2: No): ",
    "Highest education level completed (1: Never attended school or only kindergarten, 2: Grade 1-8, 3: Grade 9-11, 4: Grade 12/GED, 5: College 1-3 years, 6: College 4+ years): ",
    "Smoked at least 100 cigarettes in your entire life (1: Yes, 2: No): ",
    "Sexual orientation (1: Straight, 2: Lesbian/Gay, 3: Bisexual, 4: Other): "]
output = [float(input(i+'\n')) for i in questions]

def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros
index_list = [1, 1, 1, 1, 1, 7, 7, 9, 4, 5, 4, 5, 5, 4, 4, 3, 4, 7, 4, 6]
basis = []
for i in index_list:
    basis.append(zerolistmaker(i))
index = 0
for x in output:
    if index < 5:
        basis[index][0] = x
        index = index + 1
    elif index < 20:
        basis[index][int(x-1)] = 1
        index = index + 1
basis = [y for x in basis for y in x]

x_input = scaler.transform(np.array([basis]))
risks = [i.predict_proba(x_input) for i in clfs]
risks = [i[0][1] for i in risks]

import scipy.special
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Span, Label
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown, Button
import bokeh.io

bokeh.io.reset_output()
bokeh.io.output_notebook()

def make_plot(title, hist, edges, x, you, x_min=0, x_max=1):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, 
           bottom=0, 
           left=edges[:-1], 
           right=edges[1:], 
           fill_color="#66CCFF", 
           line_color="#66CCFF", 
           alpha=1, 
           line_alpha=0.1)
    vline = Span(location=you, dimension='height', line_color='red', line_width=3, line_alpha=0.3)
    p.renderers.extend([vline])
    p.yaxis.major_tick_line_color = None
    p.yaxis.minor_tick_line_color = None
    p.xaxis.major_tick_line_color = None
    p.xaxis.minor_tick_line_color = None
    p.xaxis.axis_label_text_font_size = "8pt"
    p.y_range.start = 0
    p.x_range.start = x_min
    p.x_range.end = x_max
    p.xaxis.axis_label = 'Risk Level'
    p.grid.grid_line_color="white"
    return p

#probas: list of predicted probabilities
#diseases: list of diseases
#yous: list of where the red lines are
#ncols: number of columns
def fancy_histogram(probas, diseases, yous, ncols, log_y=False, x_min=0, x_max=1):
    plots = []
    idx = 0
    for proba, disease, you in zip(probas, diseases, yous):
        measured = proba
        hist, edges = np.histogram(measured, density=False, bins=50)
        hist, edges = np.histogram(measured, density=False, bins=int(np.mean(edges) * 1000))
        if log_y:
            hist = np.log(hist)
        x = np.linspace(0, 1, 1000)
        p1 = make_plot(disease, hist, edges, x, you, x_min, x_max)
        plots.append(p1)
        idx += 1
    output_notebook()
    grid = gridplot(plots,
                      ncols=ncols,
                      plot_width=900//ncols,
                      plot_height=900//ncols+20,
                      toolbar_location=None)
    show(grid)

#print("Specific risk levels for each disease: ")
#print( {disease[i]: risks[i]} for i in range(11) )
    
print('Risk levels for input demographic and distribution of total population risk level:')
    
fancy_histogram(probas, diseases, np.array(risks), ncols=4)

Weight in pounds: 
150
Height in inches: 
70
Exercise frequency each week (i.e. 3 times a week -> 3) : 
4
Weekly french fries consumption frequency (i.e. 2 times a week -> 2): 
.05
Weekly alcohol consumption(i.e. 4 times a week -> 4): 
.01
Self-assessment of your health on a scale of 1 to 5, 1 being excellent: 
2
Marital status (1: Married, 2: Divorced, 3: Widowed, 4: Separated, 5: Never married, 6: Member of an unmarried couple): 
5
Employment status (1: Employed for wages, 2: Self-employed, 3: Out of work for 1 or more years, 4: Out of work for less than 1 year, 5: A homemaker, 6: A student, 7: Retired, 8: Unable to work, 9: Retired): 
6
Pregnancy Status (1: Yes, 2: No): 
2
Metropolitan Status (1: In the center of city of an MSA, 2: Outside the center city of an MSA but inside the county containing the center city, 3: Inside a suburban county of the MSA, 5: Not in an MSA ***MSA is metropolitan statistical area (region with high population density)): 
1
Physical health status for past

Loading BokehJS ...

Risk levels for input demographic and distribution of total population risk level:


Loading BokehJS ...